In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
from huggingface_hub import logout, notebook_login
# logout()
notebook_login()

In [3]:
from datasets import load_dataset

base = load_dataset("passionMan/dataset_base")
base

README.md:   0%|          | 0.00/352 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/5.49M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3351 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 3351
    })
})

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 03-06 09:29:41 __init__.py:190] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A100-PCIE-40GB. Max memory: 39.394 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.2.12 patched 32 layers with 32 QKV layers, 0 O layers and 0 MLP layers.


In [6]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]

    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("passionMan/dataset_base", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/3351 [00:00<?, ? examples/s]

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 8,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 32,
        gradient_accumulation_steps = 1,
        warmup_steps = 30, # 50
        num_train_epochs = 3, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 50,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs/model_base",
        report_to = "none", # Use this for WandB etc
        save_steps = 100,
    ),
)

Applying chat template to train dataset (num_proc=8):   0%|          | 0/3351 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=8):   0%|          | 0/3351 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=8):   0%|          | 0/3351 [00:00<?, ? examples/s]

In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,351 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 32 | Gradient Accumulation steps = 1
\        /    Total batch size = 32 | Total steps = 315
 "-____-"     Number of trainable parameters = 9,437,184


Step,Training Loss
50,1.842000
100,1.662600
150,1.611500
200,1.601900
250,1.558200
300,1.568300


In [9]:
import json
import time
import random
from datasets import load_dataset
from collections import defaultdict

### ✅ Hugging Face에서 데이터 로드
dataset_name = "passionMan/test_dataset4"
dataset = load_dataset(dataset_name, split="test")  # 'test' split 로드

### ✅ JSONL 저장 함수 (평가 결과 저장용)
def save_to_jsonl(file_path, data):
    with open(file_path, "a", encoding="utf-8") as f: 
        f.write(json.dumps(data, ensure_ascii=False) + "\n")

### ✅ 모델 응답 생성 함수
def generate_response(instruction_text, input_text, max_new_tokens=128):
    try:
        FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

        # ✅ 모델의 최대 입력 길이 가져오기 (보통 4096 또는 2048)
        max_input_length = getattr(model.config, "max_position_embeddings", 4096)

        # ✅ 입력 토큰 길이 확인
        input_tokens = tokenizer(
            alpaca_prompt.format(instruction_text, input_text, ""), 
            return_tensors="pt"
        ).to("cuda")

        input_length = input_tokens['input_ids'].shape[1]

        # 🔥 입력이 너무 길면 최대 입력 길이에 맞게 자름
        if input_length > max_input_length:
            print(f"[WARNING] Truncating input from {input_length} to {max_input_length} tokens.")
            input_text = tokenizer.decode(input_tokens['input_ids'][0, :max_input_length], skip_special_tokens=True)

        # ✅ 생성 수행 (max_new_tokens을 적용)
        outputs = model.generate(
            **tokenizer(alpaca_prompt.format(instruction_text, input_text, ""), return_tensors="pt").to("cuda"),
            max_new_tokens=max_new_tokens,  # ✅ 생성 길이 적용
            use_cache=True
        )

        decoded_outputs = tokenizer.batch_decode(outputs)
        response_texts = [output.split("### Response:\n")[-1].strip() for output in decoded_outputs]
        return response_texts[0].replace("<|eot_id|>", "")

    except Exception as e:
        print(f"[ERROR] Exception in response generation: {str(e)}")
        return None

# ✅ 데이터 경로 설정 (결과 저장용)
output_json_path = "/data/jaesung/llm_for_diabetes/src/trial/model_response/model_output_base.jsonl"

# ✅ Task별 데이터 그룹화 (각 태스크별 0~29번 샘플 선택)
grouped_data = defaultdict(list)
for item in dataset:
    grouped_data[item["task"]].append(item)

# ✅ 성능 평가할 데이터 생성 (각 태스크별 30개만 추출)
sampled_data = []
for task, samples in grouped_data.items():
    sampled_data.extend(samples[:30])  # 최대 30개 선택

# ✅ 성능 평가 시작
start_time = time.time()
total_samples = len(sampled_data)

for idx, item in enumerate(sampled_data):
    sample_start_time = time.time()

    input_text = item.get("input", "")
    instruction = item.get("instruction", "")
    task = item.get("task", "").lower()

    # ✅ 생성할 토큰 길이 설정 (생성 토큰 수 조절)
    short_context_tasks = {"qa1", "qa2", "qa3", "nli", "re"}  # 생성 32
    medium_context_tasks = {"ie"} # 생성 128
    long_context_tasks = {"summarization", "generation", "daily_diets", "alternative_diet"}  # 생성 1024

    if task in short_context_tasks:
        max_new_tokens = 128  # ✅ 생성 길이 128
    elif task in medium_context_tasks:
        max_new_tokens = 128
    elif task in long_context_tasks:
        max_new_tokens = 1024  # ✅ 생성 길이 1024
    else:
        max_new_tokens = 128  # 기본값

    try:
        model_output = generate_response(instruction, input_text, max_new_tokens)

        if model_output is not None:
            output_data = item.copy()
            output_data.update({f"model_output_{max_new_tokens}": model_output})
            save_to_jsonl(output_json_path, output_data)
        else:
            print(f"[WARNING] Skipping sample {idx+1}/{total_samples} due to length limit or generation failure.")

    except Exception as e:
        print(f"[ERROR] Skipping sample {idx+1}/{total_samples} due to unexpected error: {str(e)}")

    elapsed_time = time.time() - start_time
    avg_time_per_sample = elapsed_time / (idx + 1) 
    remaining_samples = total_samples - (idx + 1)
    estimated_remaining_time = remaining_samples * avg_time_per_sample

    print(f"[{idx+1}/{total_samples}] Sample processed in {time.time() - sample_start_time:.2f}s, ETA: {estimated_remaining_time/60:.2f} min")

print(f"\nAll samples processed. Total time: {(time.time() - start_time)/60:.2f} min")


[1/240] Sample processed in 0.67s, ETA: 2.69 min
[2/240] Sample processed in 0.30s, ETA: 1.94 min
[3/240] Sample processed in 0.31s, ETA: 1.69 min
[4/240] Sample processed in 0.48s, ETA: 1.74 min
[5/240] Sample processed in 0.60s, ETA: 1.86 min
[6/240] Sample processed in 0.15s, ETA: 1.64 min
[7/240] Sample processed in 0.39s, ETA: 1.61 min
[8/240] Sample processed in 0.73s, ETA: 1.76 min
[9/240] Sample processed in 0.14s, ETA: 1.62 min
[10/240] Sample processed in 0.30s, ETA: 1.56 min
[11/240] Sample processed in 0.13s, ETA: 1.46 min
[12/240] Sample processed in 0.44s, ETA: 1.47 min
[13/240] Sample processed in 0.60s, ETA: 1.53 min
[14/240] Sample processed in 0.71s, ETA: 1.61 min
[15/240] Sample processed in 0.61s, ETA: 1.64 min
[16/240] Sample processed in 0.39s, ETA: 1.63 min
[17/240] Sample processed in 0.39s, ETA: 1.61 min
[18/240] Sample processed in 0.45s, ETA: 1.60 min
[19/240] Sample processed in 0.30s, ETA: 1.57 min
[20/240] Sample processed in 0.26s, ETA: 1.53 min
[21/240] 

In [16]:
# import json

# import time

# def load_json(file_path):
#     with open(file_path, "r", encoding="utf-8") as file:
#         return json.load(file)

# def save_to_jsonl(file_path, data):
#     with open(file_path, "a", encoding="utf-8") as f: 
#         f.write(json.dumps(data, ensure_ascii=False) + "\n")

# def generate_response(instruction_text, input_text, max_tokens):
#     FastLanguageModel.for_inference(model) # Enable native 2x faster inference
#     inputs = tokenizer(
#     [
#         alpaca_prompt.format(
#             instruction_text, # instruction
#             input_text, # input
#             "", # output - leave this blank for generation!
#         )
#     ], return_tensors = "pt").to("cuda")

#     outputs = model.generate(**inputs, 
#                              max_new_tokens=max_tokens, 
#                              use_cache=True,)
#     decoded_outputs = tokenizer.batch_decode(outputs)

#     response_texts = [output.split("### Response:\n")[-1].strip() for output in decoded_outputs]

#     return response_texts[0].replace("<|eot_id|>", "")

# # 실행 
# input_json_paths = ["/data/jaesung/llm_for_diabetes/src/model/unsloth/test_dataset/data_classification_test.json",
# "/data/jaesung/llm_for_diabetes/src/model/unsloth/test_dataset/data_generation_test.json"]
# output_json_path = "/data/jaesung/llm_for_diabetes/src/trial/model_response/model_output.jsonl"

# data = []
# for json_path in input_json_paths:
#     data.extend(load_json(json_path))

# start_idx = 657

# start_time = time.time()
# total_samples = len(data)
# for idx, item in enumerate(data[start_idx:], start=start_idx):
#     sample_start_time = time.time()

#     input_text = item.get("input", "")
#     instruction = item.get("instruction", "")

#     try:
#         model_output_128 = generate_response(instruction, input_text, 128)
#         model_output_1024 = generate_response(instruction, input_text, 1024)

#         # 예외 발생 시 저장하지 않음
#         if model_output_128 is not None and model_output_1024 is not None:
#             output_data = item.copy()
#             output_data.update({
#                 "model_output_128": model_output_128,
#                 "model_output_1024": model_output_1024,
#             })
#             save_to_jsonl(output_json_path, output_data)
#         else:
#             print(f"[WARNING] Skipping sample {idx+1}/{total_samples} due to generation failure.")

#     except Exception as e:
#         print(f"[ERROR] Skipping sample {idx+1}/{total_samples} due to unexpected error: {str(e)}")

#     elapsed_time = time.time() - start_time
#     avg_time_per_sample = elapsed_time / (idx + 1) 
#     remaining_samples = total_samples - (idx + 1)
#     estimated_remaining_time = remaining_samples * avg_time_per_sample

#     print(f"[{idx+1}/{total_samples}] Sample processed in {time.time() - sample_start_time:.2f}s, ETA: {estimated_remaining_time/60:.2f} min")

# print(f"\nAll samples processed. Total time: {(time.time() - start_time)/60:.2f} min")

[658/1181] Sample processed in 21.42s, ETA: 0.28 min
[659/1181] Sample processed in 15.49s, ETA: 0.49 min
[660/1181] Sample processed in 9.47s, ETA: 0.61 min
[661/1181] Sample processed in 13.25s, ETA: 0.78 min
[662/1181] Sample processed in 6.73s, ETA: 0.87 min
[663/1181] Sample processed in 7.48s, ETA: 0.96 min
[664/1181] Sample processed in 7.95s, ETA: 1.06 min
[665/1181] Sample processed in 9.28s, ETA: 1.18 min
[666/1181] Sample processed in 6.28s, ETA: 1.25 min
[667/1181] Sample processed in 7.46s, ETA: 1.35 min
[668/1181] Sample processed in 6.33s, ETA: 1.42 min
[669/1181] Sample processed in 8.95s, ETA: 1.53 min
[670/1181] Sample processed in 7.25s, ETA: 1.62 min
[671/1181] Sample processed in 7.06s, ETA: 1.70 min
[672/1181] Sample processed in 5.59s, ETA: 1.77 min
[673/1181] Sample processed in 6.52s, ETA: 1.84 min
[674/1181] Sample processed in 5.77s, ETA: 1.91 min
[675/1181] Sample processed in 6.45s, ETA: 1.98 min
[676/1181] Sample processed in 6.23s, ETA: 2.05 min
[677/1181